In [39]:
import glob

import numpy as np
import pandas as pd
from keras.models import load_model

from tqdm import tqdm

from model_tools import CNNModel
from preprocessing_tools import read_data, Preprocessor

In [4]:
file_names = [file + '\ECG.txt' for file in glob.glob('test_data/*')]

In [13]:
model_dct = {
    lead: load_model(f'models/{lead}') for lead in ['i', 'ii', 'v5', 'iii', 'avr', 'avl', 'avf']
}
predict_dct = {}
predict_files = ['5849', '6479', '8555']

# проход по файлам
for idx, file_name in enumerate(file_names):
    ecg, _ = read_data(file_name)
    predict_dct[predict_files[idx]] = {
        lead: np.empty(len(ecg)) for lead in ['i', 'ii', 'v5', 'iii', 'avr', 'avl', 'avf']
    }
    
    preprocessor = Preprocessor()
    lead_dct = preprocessor.preprocess_unseen(ecg)
    
    # проход по отдельным моделям для каждого овтедения
    for lead in ['i', 'ii', 'v5', 'iii', 'avr', 'avl', 'avf']:
        X = lead_dct[lead].reshape(lead_dct[lead].shape[0], lead_dct[lead].shape[1], 1)
        predicts = model_dct[lead].predict(X)[:, 1]
        
        prev_row = 0
        idx_arr = np.arange(len(ecg), step=266) + 1330
        idx_arr = idx_arr[:len(predicts)]
        idx_arr[-1] = len(ecg)
        
        # проход по окнам в 1 секунду и назначение лейблов (1 - бигеминия, 0 - ничего/другое)
        for i, row in enumerate(idx_arr):
            predict_dct[predict_files[idx]][lead][prev_row:row] = predicts[i]
            prev_row = row

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:08<00:00,  1.27s/it]


2669/2669 [==============================] - 12s 4ms/step


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:08<00:00,  1.26s/it]


2708/2708 [==============================] - 11s 4ms/step


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.31s/it]


2724/2724 [==============================] - 12s 4ms/step


In [41]:
for file in predict_files:
    lead_labels = np.empty(shape=(len(predict_dct[file]['i']), 0))
    
    # финальный ансамбль из 7 нейросетей по каждому отведению
    for lead in ['i', 'ii', 'v5', 'iii', 'avr', 'avl', 'avf']:
        lead_labels = np.hstack((np.round(predict_dct[file][lead]).reshape(-1, 1), lead_labels))
        
    final_preds = np.max(lead_labels, axis=1).astype(np.int8)
    pd.Series(final_preds).to_csv(f'test_labels/{file}.csv', index=False)

In [49]:
labels_5849 = pd.read_csv('test_labels/5849.csv')
labels_5849

,0
0,0
1,0
2,0
3,0
4,0
...,...
22717635,0
22717636,0
22717637,0
22717638,0
